In [1]:
import os

from rnn_utils import DiagnosesDataset, split_dataset, MYCOLLATE
from rnn_utils import train_one_epoch, eval_model


import torch
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import ParameterGrid, ParameterSampler

import numpy as np
import pandas as pd

In [2]:
# Reproducibility
np.random.seed(546)
torch.manual_seed(546)
torch.cuda.manual_seed(546)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
grouping = 'ccs'
batch_size=64

In [4]:
dataset = DiagnosesDataset('data/model_data.json',grouping)
test_size = 0.15
eval_size=0.15
eval_size_corrected = eval_size/(1-test_size)

whole_train_dataset,test_dataset = split_dataset(dataset,test_size)
train_dataset, val_dataset = split_dataset(whole_train_dataset,eval_size_corrected)

len(whole_train_dataset)
len(train_dataset)
len(val_dataset)
len(test_dataset)

whole_train_dataset = DataLoader(whole_train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
train_dataloader = DataLoader(train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))

6375

5250

1125

1124

In [8]:
next(iter(whole_train_dataset)) 

{'train_sequences': {'sequence': PackedSequence(data=tensor([[0., 1., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 1.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.]]), batch_sizes=tensor([64, 29, 19,  7,  4,  4,  3,  3,  2,  2,  1,  1]), sorted_indices=tensor([51,  7, 58, 14, 38, 15, 53, 55, 42,  9, 29, 11, 12, 24, 61, 31, 37, 41,
          28, 43, 39,  2,  3, 56,  5, 25, 26, 10,  8, 27, 30, 23, 32, 33, 34, 35,
          36, 22, 21, 20, 40, 19, 18, 17, 44, 45, 46, 47, 48, 49, 50, 16, 52, 13,
          54,  6,  4, 57,  1, 59, 60,  0, 62, 63]), unsorted_indices=tensor([61, 58, 21, 22, 56, 24, 55,  1, 28,  9, 27, 11, 12, 53,  3,  5, 51, 43,
          42, 41, 39, 38, 37, 31, 13, 25, 26, 29, 18, 10, 30, 15, 32, 33, 34, 35,
          36, 16,  4, 20, 40, 17,  8, 19, 44, 45, 46, 47, 48, 49, 50,  0, 52,  6,
          54,  7, 23, 57,  2, 59, 60, 